# Tensor

In [31]:
import torch

In [21]:
x = torch.randn([1,256,256])
x = x.squeeze()
x.shape

torch.Size([256, 256])

In [22]:
x = torch.randn([3,256,256])

In [23]:
torch.unsqueeze(x, 0).shape

torch.Size([1, 3, 256, 256])

In [24]:
x.flatten(1,2).shape

torch.Size([3, 65536])

In [25]:
x.flatten(1,2).mean(axis=1).shape

torch.Size([3])

In [26]:
x = torch.randn([1,256,256])
x.transpose(0,1).shape

torch.Size([256, 1, 256])

In [30]:
torch.manual_seed(12)
x = torch.randn([1,3])
x.max(), x.argmax()

(tensor(-0.0546), tensor(2))

## cat vs stack

In [17]:
a = torch.Tensor([1])
b = torch.Tensor([2])
torch.cat([a,b], dim=0)

tensor([1., 2.])

In [18]:
torch.stack([a,b], dim=0)

tensor([[1.],
        [2.]])

In [19]:
torch.stack([a,b], dim=1)

tensor([[1., 2.]])

# Element size

In [21]:
a.nelement()

1

In [23]:
a.element_size()  # in bytes

4

Total memory allocation:


In [24]:
a.nelement() * a.element_size()

4

# Inverse Normalizer

In [61]:
from torchvision.transforms import Normalize

```
new_x = (x - mean) / std
x = new_x * std + mean
x = (new_x + mean/std) * std
x = (new_x - mean/std) / (1/std)

inv_std = 1/(std + epsilon)
inv_mean = -mean/std = mean*inv_std
```

In [62]:
class InverseNormalize(Normalize):
    """
    Undoes the normalization and returns the reconstructed images in the input domain.
    """

    def __init__(self, normalizer):
        mean = normalizer.mean
        std = normalizer.std
        std_inv = 1 / (std + 1e-7)
        mean_inv = -mean * std_inv
        super().__init__(mean=mean_inv, std=std_inv)

    def __call__(self, tensor):
        return super().__call__(tensor.clone())

In [63]:
normalizer = Normalize(mean=torch.Tensor([0.1902, 0.2077, 0.1599]), std=torch.Tensor([0.1060, 0.1060, 0.1071]))

In [64]:
normalizer.mean

tensor([0.1902, 0.2077, 0.1599])

In [65]:
inv_normalizer = InverseNormalize(normalizer)

In [66]:
inv_normalizer

InverseNormalize(mean=tensor([-1.7943, -1.9594, -1.4930]), std=tensor([9.4340, 9.4340, 9.3371]))

In [67]:
torch.manual_seed(12)
a = torch.randn(3,2,2)
a

tensor([[[-0.2138, -1.3780],
         [-0.0546,  0.4515]],

        [[ 0.7858, -1.0884],
         [-0.5599, -0.9336]],

        [[ 0.0479, -0.0844],
         [-0.1471,  0.7590]]])

In [68]:
normalizer(a)

tensor([[[ -3.8115, -14.7940],
         [ -2.3096,   2.4655]],

        [[  5.4541, -12.2277],
         [ -7.2413, -10.7672]],

        [[ -1.0455,  -2.2806],
         [ -2.8660,   5.5942]]])

In [69]:
inv_normalizer(normalizer(a))


tensor([[[-0.2138, -1.3780],
         [-0.0546,  0.4515]],

        [[ 0.7858, -1.0884],
         [-0.5599, -0.9336]],

        [[ 0.0479, -0.0844],
         [-0.1471,  0.7590]]])

In [70]:
assert (inv_normalizer(normalizer(a)) - a < 10e-7).all()